In [1]:
import pandas as pd
import numpy as np
import json

In [3]:
data = []
with open('data/data/business.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))
df_business = pd.DataFrame(data)
df_business = df_business[['business_id', 'stars',
                            'review_count', 'is_open']]
df_business = df_business.rename(columns={'stars':'bus_avg_rate',
                                          'review_count': 'review_count_bus'})

data = []
with open('data/data/user.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))
df_user = pd.DataFrame(data)
df_user = df_user[['user_id', 'fans', 'review_count', 'average_stars']]
df_user = df_user.rename(columns={'review_count':'review_count_user',
                                  'average_stars':'user_avg_rate'})

data = []
with open('data/data/train_review_ratings.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))
df_train_ratings = pd.DataFrame(data)

In [4]:
df_train_ratings.head()

,user_id,business_id,stars
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0
2,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0
3,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0
4,vrKkXsozqqecF3CW4cGaVQ,rjuWz_AD3WfXJc03AhIO_w,5.0


In [5]:
df_business.head()

,business_id,bus_avg_rate,review_count_bus,is_open
0,MTSW4McQd7CbVtyjqoe9mw,4.0,80,1
1,UJsufbvfyfONHeWdvAHKjA,3.5,6,1
2,0bPLkL0QhhPO5kt1_EXmNQ,4.5,100,0
3,MUTTqe8uqyMdBl186RmNeA,4.0,245,1
4,ROeacJQwBeh05Rqg7F6TCg,4.5,205,1


In [6]:
df_user.head()

,user_id,fans,review_count_user,user_avg_rate
0,qVc8ODYU5SZjKXVBgXdI7w,267,585,3.91
1,j14WgRoU_-2ZE1aw1dXrJg,3138,4333,3.74
2,SZDeASXq7o05mMNLshsdIA,28,224,4.27
3,q_QQ5kBBwlCcbL1s4NVK3g,1357,1221,3.85
4,1McG5Rn_UDkmlkZOrsdptg,1,7,4.29


In [43]:
df_merged = pd.merge(df_train_ratings, df_user, on='user_id', how='inner')
df_merged = pd.merge(df_merged, df_business, on='business_id', how='inner')

X = df_merged.drop('stars', axis=1)
y = df_merged['stars']

# X['user_id_hashed'] = X['user_id'].apply(lambda x: hash(x))
# X.drop(['user_id'], axis=1, inplace=True)
# X = X.rename(columns={'user_id_hashed':'user_id'})

# X['business_id_hashed'] = X['business_id'].apply(lambda x: hash(x))
# X.drop(['business_id'], axis=1, inplace=True)
# X = X.rename(columns={'business_id_hashed':'business_id'})

In [44]:
import hashlib

def hash_id(id_str, num_bins):
    hash_obj = hashlib.md5(id_str.encode())
    return int(hash_obj.hexdigest(), 16) % num_bins

num_bins = 1000  # Adjust the number of bins as needed
X['user_id_hashed'] = X['user_id'].apply(lambda x: hash_id(x, num_bins))
X.drop(['user_id'], axis=1, inplace=True)
X = X.rename(columns={'user_id_hashed':'user_id'})

X['business_id_hashed'] = X['business_id'].apply(lambda x: hash_id(x, num_bins))
X.drop(['business_id'], axis=1, inplace=True)
X = X.rename(columns={'business_id_hashed':'business_id'})


In [45]:
data = []
with open('data/data/val_review_ratings.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

df_val_ratings = pd.DataFrame(data)
df_val_ratings = pd.merge(df_val_ratings, df_user, on='user_id', how='inner')
df_val_ratings = pd.merge(df_val_ratings, df_business, on='business_id', how='inner')
X_val = df_val_ratings.drop('stars', axis=1)
y_val = df_val_ratings['stars']

In [46]:
import hashlib

def hash_id(id_str, num_bins):
    hash_obj = hashlib.md5(id_str.encode())
    return int(hash_obj.hexdigest(), 16) % num_bins

num_bins = 1000

X_val['user_id_hashed'] = X_val['user_id'].apply(lambda x: hash_id(x, num_bins))
X_val.drop(['user_id'], axis=1, inplace=True)
X_val = X_val.rename(columns={'user_id_hashed': 'user_id'})

X_val['business_id_hashed'] = X_val['business_id'].apply(lambda x: hash_id(x, num_bins))
X_val.drop(['business_id'], axis=1, inplace=True)
X_val = X_val.rename(columns={'business_id_hashed': 'business_id'})


In [48]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# X_val = scaler.fit_transform(X_val)
X.drop('year', axis=1, inplace=True)
X_val.drop('year', axis=1, inplace=True)

In [51]:
X.head()

,fans,review_count_user,user_avg_rate,bus_avg_rate,review_count_bus,is_open,user_id,business_id
0,0,33,4.06,3.0,169,1,103,799
1,2,39,3.05,3.0,169,1,260,799
2,7,490,3.33,3.0,169,1,618,799
3,39,483,3.46,3.0,169,1,22,799
4,2,128,3.12,3.0,169,1,44,799


In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()

lin_reg.fit(X, y)
y_pred = lin_reg.predict(X_val)
RMSE = np.sqrt(mean_squared_error(y_val, y_pred))
print(RMSE)

0.7393953280153174


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
lin_reg = LinearRegression()

pca = PCA()

param_grid = {
    'pca__n_components': [2, 3, 4],  # Number of principal components to consider
    'lin_reg__fit_intercept': [True, False],  # Whether to calculate the intercept for this model
    'lin_reg__positive': [True, False],  # Whether to normalize the features before regression
}

from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('pca', pca),
    ('lin_reg', lin_reg)
])

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(X, y)
best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(X_val)
RMSE = np.sqrt(mean_squared_error(y_val, y_pred))
print(RMSE)

ValueError: 
All the 60 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/home/mouhamadali/anaconda3/envs/ml_book_env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/mouhamadali/anaconda3/envs/ml_book_env/lib/python3.10/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/mouhamadali/anaconda3/envs/ml_book_env/lib/python3.10/site-packages/sklearn/pipeline.py", line 416, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/mouhamadali/anaconda3/envs/ml_book_env/lib/python3.10/site-packages/sklearn/pipeline.py", line

In [ ]:
from sklearn.svm import LinearSVR

svr_reg_linear = LinearSVR()
param_grid_linear = [
    {'C': [0.1, 1]}
]
grid_search_linear = GridSearchCV(svr_reg_linear, param_grid_linear, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search_linear.fit(X, y)
best_estimator_linear = grid_search_linear.best_estimator_
print("Best estimator (linear kernel):", best_estimator_linear)

y_pred = best_estimator_linear.predict(X_val)
RMSE = np.sqrt(mean_squared_error(y_val, y_pred))
print(RMSE)

In [ ]:
svr_reg_rbf = SVR(kernel='rbf')
param_grid_rbf = [
    {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}
]
grid_search_rbf = GridSearchCV(svr_reg_rbf, param_grid_rbf, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search_rbf.fit(X, y)
best_estimator_rbf = grid_search_rbf.best_estimator_
print("Best estimator (rbf kernel):", best_estimator_rbf)

In [ ]:
y_pred = best_estimator_rbf.predict(X_val)
RMSE = np.sqrt(mean_squared_error(y_val, y_pred))
print(RMSE)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(X, y)
grid_search.best_params_
grid_search.best_estimator_